In [2]:
from datetime import timedelta
from glob import glob
from os import mkdir, remove
from os.path import join, exists

import numpy as np

from ltbio.biosignals.modalities import EEG
from ltbio.biosignals.timeseries import Timeline
from ltbio.processing.formaters import Segmenter

In [3]:
common_path = "/Volumes/MMIS-Saraiv/Datasets/Sapienza/denoised_biosignal/"
out_common_path = "/Volumes/MMIS-Saraiv/Datasets/Sapienza/denoised_txt_epochs/"
all_files = glob(join(common_path, "*.biosignal"))

In [4]:
channel_order = ('C3', 'C4', 'Cz', 'F3', 'F4', 'F7', 'F8', 'Fp1', 'Fp2', 'Fpz', 'Fz', 'O1', 'O2', 'P3', 'P4', 'Pz', 'T3', 'T4', 'T5', 'T6')  # without mastoids

In [5]:
for filepath in all_files:
    filename = filepath.split('/')[-1].split('.')[0]
    print(filename)
    subject_out_path = join(out_common_path, filename)
    if not exists(subject_out_path):
        mkdir(subject_out_path)
    # Load Biosignal
    x = EEG.load(filepath)
    x = x[channel_order]  # get only channels of interest

    # Get only signal with quality
    good = Timeline.load(join(common_path, filename + '_good.timeline'))
    x = x[good]
    
    # Epochs
    print("Segmenting...")
    segmenter = Segmenter(timedelta(seconds=2))
    segmented_x = segmenter(x)
    print("Segmented")
    
    # Go by segment
    intervals = segmented_x['O2'].domain
    for i, interval in enumerate(intervals):
        print(f"Segment {i}")
        segment = segmented_x[interval]
        _array: np.ndarray = segment.to_array(channel_order).T
        # Make a txt in ASCII format, and put the array data there; write with maximum 4 decimal places
        with open(join(subject_out_path, f"{filename}_{i}.txt"), 'w', encoding='ascii') as f:
            np.savetxt(f, _array, fmt='%.4f', delimiter='\t')
         

In [10]:
out_common_path = "/Volumes/MMIS-Saraiv/Datasets/Sapienza/roma_protocol_source/"
all_participant_txt_directories = glob(join(out_common_path, "PARTICIPANT*"))
for directory in all_participant_txt_directories:
    all_txt_files = glob(join(directory, "*.txt"))
    sub_filename = directory.split('/')[-1]
    all_epoch_numbers = [int(file.split('/')[-1].split('_')[-1].split('.')[0]) for file in all_txt_files]
    all_epoch_numbers.sort()
    # delete last file
    last_epoch_number = all_epoch_numbers[-1]
    last_file = join(directory, f"{sub_filename}_{last_epoch_number}.txt")
    print(last_file)
    remove(last_file)

In [3]:
out_common_path = "/Volumes/MMIS-Saraiv/Datasets/Sapienza/roma_protocol_source"
all_participant_txt_directories = glob(join(out_common_path, "PARTICIPANT*"))
for directory in all_participant_txt_directories:
    all_txt_files = glob(join(directory, "*.txt"))
    for file in all_txt_files:
        # read with numpy
        data = np.loadtxt(file, delimiter='\t')
        # are there nans?
        if np.isnan(data).any():
            print(file)
            #remove(file)